In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('detected.csv', index_col=0)
df

,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,fire hydrant,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy bear,hair drier,toothbrush
person,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,8,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0


In [4]:
#columns = df.columns.to_list()
#scaler = StandardScaler()
#df = scaler.fit_transform(df.values)
#df = pd.DataFrame(df, columns=columns)
#df

In [5]:
#from pyod.models.ecod import ECOD
#from pyod.models.knn import KNN
from pyod.models.iforest import IForest

df_results_pred = pd.DataFrame()
df_results_scores = pd.DataFrame()

models_dict = {
    #'ECOD': ECOD(),
    #'KNN': KNN(),
    'IForest': IForest(),
}

for model_name, model in models_dict.items():
    print(f'Fitting {model_name}...')
    model.fit(df.values)
    predictions = model.predict(df.values)
    scores = model.decision_scores_
    df_results_pred[model_name+"_p"] = predictions
    df_results_scores[model_name+"_s"] = scores

Fitting IForest...


In [6]:
# get the number of outliers (where all models agree)
df_results_pred['sum'] = df_results_pred.sum(axis=1)
df_results_pred['outlier'] = df_results_pred['sum'] == len(models_dict)
df_results_pred

,IForest_p,sum,outlier
0,0,0,False
1,0,0,False
2,0,0,False
3,0,0,False
4,0,0,False
...,...,...,...
1926,0,0,False
1927,0,0,False
1928,0,0,False
1929,0,0,False


In [7]:
df_results_pred["sum"].value_counts()

sum
0    1742
1     189
Name: count, dtype: int64

In [8]:
# normalize df with min-max normalization (esclude label column)
df_results_scores = (df_results_scores - df_results_scores.min()) / (df_results_scores.max() - df_results_scores.min())
# fill NaN values with 0
df_results_scores = df_results_scores.fillna(0)
df_results_scores["average_score"] = df_results_scores.mean(axis=1)
df_results_scores["average_score"] = df_results_scores["average_score"].round(3)
df_results_scores

,IForest_s,average_score
0,0.000000,0.000
1,0.003146,0.003
2,0.000000,0.000
3,0.000000,0.000
4,0.000000,0.000
...,...,...
1926,0.015915,0.016
1927,0.015915,0.016
1928,0.036537,0.037
1929,0.015915,0.016


In [9]:
outliers_index = df_results_pred[df_results_pred['outlier'] == True].index
df_results_scores = df_results_scores.loc[outliers_index]
df_results_scores

,IForest_s,average_score
149,0.610511,0.611
150,0.610511,0.611
151,0.677352,0.677
180,0.624012,0.624
200,0.637699,0.638
...,...,...
1768,0.704285,0.704
1769,0.656898,0.657
1770,0.613530,0.614
1771,0.704285,0.704


In [10]:
# plot the average score for each label with plotly
fig = px.scatter(df_results_scores, x=df_results_scores.index,
                  y="average_score",
                  title="Average score for each label", size="average_score", size_max=20, opacity=0.5)
fig.show()

In [19]:
df.iloc[1539][df.iloc[1539]!=0]

chair     2
remote    3
book      7
Name: 1, dtype: int64

In [11]:
# create movin average column of average_score
df_results_scores['mov_avg'] = df_results_scores['average_score'].rolling(50).mean()
df_results_scores

,IForest_s,average_score,mov_avg
149,0.610511,0.611,NaN
150,0.610511,0.611,NaN
151,0.677352,0.677,NaN
180,0.624012,0.624,NaN
200,0.637699,0.638,NaN
...,...,...,...
1768,0.704285,0.704,0.68922
1769,0.656898,0.657,0.68848
1770,0.613530,0.614,0.68584
1771,0.704285,0.704,0.68414


In [12]:
# plot the moving average score for each label with plotly using line chart
fig = px.line(df_results_scores.iloc[50:], x=df_results_scores.iloc[50:].index,
                  y="mov_avg",
                  title="Moving average score for each label")
fig.show()

In [13]:
# save model to pickle
import pickle
filename = 'iforest_model_detected.pkl'
with open(filename, 'wb') as file:
    pickle.dump(models_dict['IForest'], file)
#pickle.dump(models_dict['IForest'], open(filename, 'wb'))